In [1]:
!pip install transformers
import nltk
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [2]:
import requests
import random
import re
from bs4 import BeautifulSoup
from nltk import word_tokenize, WordNetLemmatizer


def lemmatize_query(query):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(query)
    lemmatized_query = ' '.join([lemmatizer.lemmatize(token) for token in tokens])
    return lemmatized_query


def Scraper(yourquery):
    if " " in yourquery:
        yourquery = lemmatize_query(yourquery.replace(" ", "+"))

    print("Lemmatized Query:"+yourquery)

    url = "https://search.wikileaks.org/?q={}".format(yourquery)

    ua_list = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19577",
        "Mozilla/5.0 (X11) AppleWebKit/62.41 (KHTML, like Gecko) Edge/17.10859 Safari/452.6",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2656.18 Safari/537.36",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML like Gecko) Chrome/44.0.2403.155 Safari/537.36",
        "Mozilla/5.0 (Linux; U; en-US) AppleWebKit/525.13 (KHTML, like Gecko) Chrome/0.2.149.27 Safari/525.13",
        "Mozilla/5.0 (Windows; U; Windows NT 6.0; en-US) AppleWebKit/533.20.25 (KHTML, like Gecko) Version/5.0.4 Safari/533.20.27",
        "Mozilla/5.0 (Macintosh; U; PPC Mac OS X 10_5_8; zh-cn) AppleWebKit/533.20.25 (KHTML, like Gecko) Version/5.0.4 Safari/533.20.27"
    ]
    ua = random.choice(ua_list)
    headers = {'User-Agent': ua}

    request = requests.get(url=url, headers=headers)
    content = request.text

    def findlinks(content):
        soup = BeautifulSoup(content, 'html.parser')
        results = soup.find_all('div', class_='result')
        links = []
        for result in results:
            link = result.find('a', href=True)
            if link:
                links.append(link['href'])
        return links

    if request.status_code == 200:
        print("Request went through.\n")
        return findlinks(content)


# **Gather Data**

In [5]:
import requests
from bs4 import BeautifulSoup
import nltk
import random
import csv
from collections import OrderedDict

def sanitize_string(text):
    # Define the characters you want to remove or replace
    characters_to_remove = ['\n', '\r', '\t', '"', ",", '\\']  # Newlines, carriage returns, and tabs

    # Replace problematic characters with a space
    replacement_character = ' '

    # Remove or replace the problematic characters
    for char in characters_to_remove:
        text = text.replace(char, replacement_character)

    return text

def get_page_content(links):
    ua_list = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19577",
        "Mozilla/5.0 (X11) AppleWebKit/62.41 (KHTML, like Gecko) Edge/17.10859 Safari/452.6",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2656.18 Safari/537.36",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML like Gecko) Chrome/44.0.2403.155 Safari/537.36",
        "Mozilla/5.0 (Linux; U; en-US) AppleWebKit/525.13 (KHTML, like Gecko) Chrome/0.2.149.27 Safari/525.13",
        "Mozilla/5.0 (Windows; U; Windows NT 6.0; en-US) AppleWebKit/533.20.25 (KHTML, like Gecko) Version/5.0.4 Safari/533.20.27",
        "Mozilla/5.0 (Macintosh; U; PPC Mac OS X 10_5_8; zh-cn) AppleWebKit/533.20.25 (KHTML, like Gecko) Version/5.0.4 Safari/533.20.27"
    ]
    ua = random.choice(ua_list)
    headers = {'User-Agent': ua}

    all_data = OrderedDict()  # Use OrderedDict to preserve the order of links and data

    for url in links:
        try:
            # Send an HTTP GET request to the URL with the User-Agent header
            response = requests.get(url, headers=headers, timeout=5)  # Set a timeout of 5 seconds
            response.raise_for_status()  # Check if the request was successful
            page_content = response.text

            print('['+url+"] FETCHING...")

            # Parse the HTML content using Beautiful Soup
            soup = BeautifulSoup(page_content, 'html.parser')
            doc_description = soup.find('div', {'id': 'doc-description'})

            if doc_description:
                print('[' + url + "] SUCCESS!")
                page_title = sanitize_string(soup.title.text)
                doc_description_text = sanitize_string(doc_description.get_text().strip())
                all_data[url] = (page_title, url, doc_description_text)

        except requests.exceptions.RequestException as e:
            print(f"Error fetching the page for URL: {url}\nError details: {e}")
            continue  # Move to the next link if an exception occurs

    return all_data.values()  # Return the values of the OrderedDict

if __name__ == "__main__":
    # Replace this with the list of links you want to fetch
    nltk.download('punkt')
    query = input("Insert the topic of your query:")
    links = Scraper(query)
    print(links)

    fetched_data = get_page_content(links)

    if fetched_data:
        # Save the data to a CSV file
        with open('page_data.csv', mode='w', encoding='utf-8', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Title', 'URL', 'Content'])
            writer.writerows(fetched_data)

        print("Data saved to 'page_data.csv' successfully.")

    else:
        print("Failed to fetch content for all the links.")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Insert the topic of your query:area 51
Lemmatized Query:area+51
Request went through.

['https://wikileaks.org/berats-box/emailid/27744', 'https://wikileaks.org/gifiles/docs/16/1652778_area-51-vets-break-silence-sorry-but-no-space-aliens-.html', 'https://wikileaks.org/gifiles/docs/38/382259_fw-ct-area-51-vets-break-silence-sorry-but-no-space-aliens-.html', 'https://wikileaks.org/gifiles/docs/23/2310498_-military-the-very-secret-history-of-area-51-.html', 'https://wikileaks.org/gifiles/docs/15/1543658_-tactical-the-very-secret-history-of-area-51-.html', 'https://wikileaks.org/gifiles/docs/37/376282_coastzone-area-51-revisited-.html', 'https://wikileaks.org/gifiles/docs/18/1897445_-tactical-npr-org-area-51-uncensored-was-it-ufos-or-the-ussr.html', 'https://wikileaks.org/gifiles/docs/18/1895081_-tactical-windows-1252-q-area_51.html', 'https://wikileaks.org/gifiles/docs/23/2320294_-military-windows-1252-q-area_51.html', 'https://wikileaks.org/plusd/cables/1979MBFRV00636_e.html', 'https://w

# **Answer Questions**

In [6]:
import pandas as pd
from transformers import BertTokenizer, BertForQuestionAnswering
from sklearn.metrics.pairwise import cosine_similarity
import torch
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Check if GPU is available and set the device accordingly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

# Replace 'page_data.csv' with the name of your CSV file
csv_file = 'page_data.csv'
question = input("Insert your question:")
# Download stopwords and initialize WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

try:
    # Read the CSV file into a Pandas DataFrame
    df = pd.read_csv(csv_file)

    # Remove rows with empty 'Link'
    df.dropna(subset=['URL'], inplace=True)

    # Preprocess the content for BERT question answering
    tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

    def preprocess_content(row):
        context = row['Content']
        inputs = tokenizer(question, context, return_tensors='pt', max_length=512, truncation=True, padding='max_length')
        return inputs

    # Apply the preprocessing function to each row
    df['inputs'] = df.apply(preprocess_content, axis=1)

    # Load the pre-trained BERT model for question answering
    model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
    model.to(device)

    # Function to perform question answering
    def question_answering(inputs):
        inputs.to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            start_logits, end_logits = outputs.start_logits, outputs.end_logits

        # Get the start and end positions with the highest probability
        start_index = torch.argmax(start_logits, dim=1).item()
        end_index = torch.argmax(end_logits, dim=1).item()

        # Get the answer span from the context
        context_tokens = inputs['input_ids'][0]
        answer = tokenizer.decode(context_tokens[start_index:end_index+1], skip_special_tokens=True)

        return answer

    # Perform question answering for each row in the DataFrame
    answer_list = []
    for _, row in df.iterrows():
        answer = question_answering(row['inputs'])
        answer_list.append(answer)

    # Calculate cosine similarity and append the results to the DataFrame
    df['Answer'] = answer_list
    df['Cosine Similarity with Question (Content)'] = None
    df['Cosine Similarity with Question (Answer)'] = None
    df['Cosine Similarity with Question (Title)'] = None

    for i, row in df.iterrows():
        content = row['Content']
        title = row['Title']
        answer = row['Answer']

        # Preprocess question, content, and title (remove stop words and lemmatize)
        question = ' '.join([lemmatizer.lemmatize(word) for word in question.split() if word.lower() not in stop_words])
        content = ' '.join([lemmatizer.lemmatize(word) for word in content.split() if word.lower() not in stop_words])
        title = ' '.join([lemmatizer.lemmatize(word) for word in title.split() if word.lower() not in stop_words])

        embeddings = tokenizer([question, content, answer, title], return_tensors='pt', padding=True, truncation=True)
        embeddings.to(device)
        question_embedding = model.base_model(**embeddings)['last_hidden_state'][:, 0].detach().cpu().numpy()
        content_embedding = model.base_model(**embeddings)['last_hidden_state'][:, 1].detach().cpu().numpy()
        answer_embedding = model.base_model(**embeddings)['last_hidden_state'][:, 2].detach().cpu().numpy()
        title_embedding = model.base_model(**embeddings)['last_hidden_state'][:, 3].detach().cpu().numpy()

        similarity_question_content = cosine_similarity(question_embedding, content_embedding)[0][0]
        similarity_question_answer = cosine_similarity(question_embedding, answer_embedding)[0][0]
        similarity_question_title = cosine_similarity(question_embedding, title_embedding)[0][0]

        df.at[i, 'Cosine Similarity with Question (Content)'] = similarity_question_content
        df.at[i, 'Cosine Similarity with Question (Answer)'] = similarity_question_answer
        df.at[i, 'Cosine Similarity with Question (Title)'] = similarity_question_title

    # Save the DataFrame to a result CSV file
    result_csv_file = 'result_data.csv'
    df.to_csv(result_csv_file, index=False)
    print(f"DataFrame saved to '{result_csv_file}' successfully.")

    # Sort the DataFrame based on average cosine similarity and print the top 5 answers
    df['Average Cosine Similarity'] = df[
        ['Cosine Similarity with Question (Content)', 'Cosine Similarity with Question (Answer)',
         'Cosine Similarity with Question (Title)']].mean(axis=1)
    sorted_df = df.sort_values(by='Average Cosine Similarity', ascending=False)
    top_5_answers = sorted_df.head(5)
    pd.set_option('display.max_colwidth', None)  # Set the option to display the entire answer
    print("\nTop 5 Answers with Highest Average Cosine Similarity:")
    print(top_5_answers[['URL', 'Answer', 'Average Cosine Similarity']])

except FileNotFoundError:
    print(f"Error: The file '{csv_file}' not found.")
except pd.errors.EmptyDataError:
    print(f"Error: The file '{csv_file}' is empty or contains invalid data.")
except pd.errors.ParserError:
    print(f"Error: Unable to parse the file '{csv_file}'. Check if it's a valid CSV file.")
except Exception as e:
    print(f"Error: {e}")


cuda
Insert your question:are there aliens in area 51?


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-

DataFrame saved to 'result_data.csv' successfully.

Top 5 Answers with Highest Average Cosine Similarity:
                                                                                                             URL  \
0       https://wikileaks.org/gifiles/docs/16/1652778_area-51-vets-break-silence-sorry-but-no-space-aliens-.html   
1  https://wikileaks.org/gifiles/docs/38/382259_fw-ct-area-51-vets-break-silence-sorry-but-no-space-aliens-.html   
4                                 https://wikileaks.org/gifiles/docs/37/376282_coastzone-area-51-revisited-.html   
2               https://wikileaks.org/gifiles/docs/23/2310498_-military-the-very-secret-history-of-area-51-.html   
3               https://wikileaks.org/gifiles/docs/15/1543658_-tactical-the-very-secret-history-of-area-51-.html   

                                                                                    Answer  \
0                                                                sorry but no space aliens   
1        